# Importing Required Libraries


In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import cv2
import seaborn as sns
import glob
import xml.etree.ElementTree as ET
from PIL import Image
import os
import shutil
import torch

Importing Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Dataset


The dataset we are using is a mix of ...

Naming structure for the files is as follows,

04-P

xx - n

xx- Image number
n - Uploader (G - Ginnie, P - Pocholo)





## Dataset Preparation

Name directories inside google drive

In [6]:
# Google Drive Version
# label_dir = "/content/drive/MyDrive/IAT481/CV_Proj/labels/"
# images_dir = "/content/drive/MyDrive/IAT481/CV_Proj/images/"

# Local Version
label_dir = "C:\\Users\\Pocholo\\Desktop\\481\\CV_Project\\labels\\"
images_dir = "C:\\Users\\Pocholo\\Desktop\\481\\CV_Project\\images\\"

In [10]:
# I ran this once to clean up file names (Remove label studio unique ID)
#
# for file in os.listdir(label_dir):
#     if file.endswith(".txt"):
#         test = file.split("-")
#         oldName = label_dir + file;
#         newName = label_dir + test[1] + "-" + test[2]
#         os.rename(oldName,newName)
#         # print(file)

# for file in os.listdir(images_dir):
#     if file.endswith(".jpg"):
#         test = file.split("-")
#         oldName = images_dir + file;
#         newName = images_dir + test[1] + "-" + test[2]
#         os.rename(oldName, newName)

for file in os.listdir(images_dir):
    print(file)

for file in os.listdir(label_dir):
    print(file)

01-G.jpg
01-P.jpg
02-G.jpg
02-P.jpg
03-G.jpg
03-P.jpg
04-G.jpg
04-P.jpg
05-G.jpg
05-P.jpg
06-G.jpg
06-P.jpg
07-G.jpg
07-P.jpg
08-G.jpg
08-P.jpg
09-G.jpg
09-P.jpg
10-G.jpg
10-P.jpg
11-G.jpg
11-p.jpg
12-G.jpg
12-p.jpg
13-G.jpg
13-p.jpg
14-G.jpg
14-P.jpg
15-G.jpg
15-P.jpg
16-G.jpg
16-P.jpg
17-G.jpg
17-P.jpg
18-G.jpg
18-P.jpg
19-G.jpg
19-P.jpg
20-G.jpg
20-P.jpg
21-G.jpg
21-P.jpg
22-G.jpg
22-P.jpg
23-G.jpg
23-P.jpg
24-G.jpg
24-P.jpg
25-G.jpg
25-P.jpg
26-G.jpg
26-P.jpg
27-G.jpg
27-P.jpg
28-G.jpg
28-P.jpg
29-G.jpg
29-P.jpg
30-G.jpg
30-P.jpg
01-G.txt
01-P.txt
02-G.txt
02-P.txt
03-G.txt
03-P.txt
04-G.txt
04-P.txt
05-G.txt
05-P.txt
06-G.txt
06-P.txt
07-G.txt
07-P.txt
08-G.txt
08-P.txt
09-G.txt
09-P.txt
10-G.txt
10-P.txt
11-G.txt
11-p.txt
12-G.txt
12-p.txt
13-G.txt
13-p.txt
14-G.txt
14-P.txt
15-G.txt
15-P.txt
16-G.txt
16-P.txt
17-G.txt
17-P.txt
18-G.txt
18-P.txt
19-G.txt
19-P.txt
20-G.txt
20-P.txt
21-G.txt
21-P.txt
22-G.txt
22-P.txt
23-G.txt
23-P.txt
24-G.txt
24-P.txt
25-G.txt
25-P.txt
26-G.txt
2

Next step is to split the data into training and valuation,

Per instructions we need to use 20/10 split for each dataset, to achieve this I used every picture divisible by 3 (30 images = 10 per person)

In [7]:
images_dir_train = "C:\\Users\\Pocholo\\Desktop\\481\\CV_Project\\images\\train\\"
images_dir_val = "C:\\Users\\Pocholo\\Desktop\\481\\CV_Project\\images\\val\\"

label_dir_train = "C:\\Users\\Pocholo\\Desktop\\481\\CV_Project\\labels\\train\\"
label_dir_val = "C:\\Users\\Pocholo\\Desktop\\481\\CV_Project\\labels\\val\\"


In [ ]:
for file in os.listdir(images_dir):
    if file.endswith(".jpg") and int(file.split("-")[0]) % 3 == 0:
        os.rename(images_dir + file, images_dir_val + file)
    else:
        os.rename(images_dir + file, images_dir_train + file)
        
for file in os.listdir(label_dir):
    if file.endswith(".txt") and int(file.split("-")[0]) % 3 == 0:
        os.rename(label_dir + file, label_dir_val + file)
    else:
        os.rename(label_dir + file, label_dir_train + file)

In [9]:
df = {'name': [],
      'label': [],
      'width': [],
      'height': [],
      'x': [],
      'y': [],
     }

test = {'holder': []}

for file in os.listdir(label_dir_train):
    if file.endswith(".txt"):
        content = open(label_dir_train + file)
        readContent = content.readlines()
        for line in readContent:
            clean = line.split(" ")
            df['name'].append(file)
            df['label'].append(clean[0])
            df['x'].append(clean[1])
            df['y'].append(clean[2])
            df['width'].append(clean[3])
            df['height'].append(clean[4])



val = pd.DataFrame(df)
val


,name,label,width,height,x,y
0,01-G.txt,5,0.345558272208639,0.2616136919315404\n,0.5639771801140994,0.2506112469437653
1,01-G.txt,1,0.3162184189079055,0.22982885085574584\n,0.47758761206193967,0.6821515892420539
2,01-P.txt,3,0.11648,0.22200000000000003\n,0.09984,0.505
3,01-P.txt,2,0.13624000000000006,0.26\n,0.23868000000000003,0.496
4,01-P.txt,0,0.10608000000000004,0.1979999999999999\n,0.37439999999999996,0.49700000000000005
...,...,...,...,...,...,...
5588,toonie97_jpeg.rf.993d5be04a3903817914d36aca11e...,5,1,0.9671875,0.5,0.48359375
5589,toonie97_jpeg.rf.a24ff2bf90f52ded6695e3bf63b8c...,5,0.94375,0.9171875,0.471875,0.45859375
5590,toonie9_jpeg.rf.150e384a229f4e3b39dfb760fec311...,5,0.94921875,0.92890625,0.475,0.4765625
5591,toonie9_jpeg.rf.4b2fee7a8a37268dfb557fd6150ed9...,5,1,1,0.5,0.5


# YOLO Training


## Required imports to run GPU

In [2]:
import ultralytics
from ultralytics import YOLO

In [3]:
model = YOLO('yolov8s.pt')

In [4]:
%%writefile config.yaml

# Google Drive
# path: /content/drive/MyDrive/IAT481/CV_Proj/
# train: /content/drive/MyDrive/IAT481/CV_Proj/images/
# val: /content/drive/MyDrive/IAT481/CV_Proj/images/

# Local
path: C:\\Users\\Pocholo\\Desktop\\481\\CV_Project\\
train: C:\\Users\\Pocholo\\Desktop\\481\\CV_Project\\images\\train\\
val: C:\\Users\\Pocholo\\Desktop\\481\\CV_Project\\images\\val\\

# Classes
nc: 6

# Class Names
names:
  0: Dime
  1: Loonie
  2: Nickel
  3: Penny
  4: Quarter
  5: Toonie

Overwriting config.yaml


In [3]:
import torch

In [5]:
torch.cuda.is_available()

True

In [6]:
torch.cuda.current_device()
torch.cuda.get_device_capability()

(8, 6)

In [7]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 3070'

## Baseline Stock model


As part of testing, we ran the stock model without any edits except EPOCH = 150

In [4]:
model.train(data="config.yaml",epochs=150,patience=5,batch=8, lr0=0.0005,imgsz=640)
model.to('cuda')

Ultralytics YOLOv8.1.27 🚀 Python-3.10.13 torch-2.2.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3070, 8192MiB)
engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=config.yaml, epochs=150, time=None, patience=5, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_box

train: Scanning C:\Users\Pocholo\Desktop\481\CV_Project\labels\train.cache... 1522 images, 129 backgrounds, 0 corrupt: 100%|██████████| 1522/1522 [00:00<?, ?it/s]
val: Scanning C:\Users\Pocholo\Desktop\481\CV_Project\labels\val... 118 images, 9 backgrounds, 0 corrupt: 100%|██████████| 118/118 [00:00<00:00, 590.01it/s]

val: New cache created: C:\Users\Pocholo\Desktop\481\CV_Project\labels\val.cache


Plotting labels to runs\detect\train\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.0005' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train
Starting training for 150 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/150      2.36G      1.364      2.613      1.561          4        640: 100%|██████████| 191/191 [00:21<00:00,  8.97it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:01<00:00,  6.67it/s]

...

                   all        118        444      0.754       0.75      0.821      0.531
Stopping training early as no improvement observed in last 5 epochs. Best results observed at epoch 29, best model saved as best.pt.
To update EarlyStopping(patience=5) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



34 epochs completed in 0.197 hours.
Optimizer stripped from runs\detect\train\weights\last.pt, 22.5MB
Optimizer stripped from runs\detect\train\weights\best.pt, 22.5MB

Validating runs\detect\train\weights\best.pt...
Ultralytics YOLOv8.1.27 🚀 Python-3.10.13 torch-2.2.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3070, 8192MiB)
Model summary (fused): 168 layers, 11127906 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:01<00:00,  6.84it/s]


                   all        118        444      0.786      0.832      0.887      0.567
                  Dime        118         50      0.662      0.745      0.789      0.405
                Loonie        118         56      0.798      0.982      0.949      0.659
                Nickel        118         61      0.691      0.787      0.861      0.593
                 Penny        118         83      0.837      0.864      0.918      0.638
               Quarter        118        114      0.842      0.652       0.85      0.451
                Toonie        118         80      0.888      0.963      0.959      0.655
Speed: 0.4ms preprocess, 3.5ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs\detect\train


YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(96, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_s

# Model Optimization

https://docs.ultralytics.com/guides/hyperparameter-tuning/#introduction

Using the docs as the guide we decided to tune the model on the commonly used hyperparameters

To tune the model, we adjusted things like lr0, batch sizes, and epochs 

The most important adjustment is related to lr0, and we increased the lr0 from 0.0005 to 0.5

However, we soon found out that YOLO automatically ignore lr0 tuning for any datasets that are on the smaller side  and automatically applies adamW and lr_fit and calculates the lr0 best suited for us

So for hyperparameters, we will be changing batch size

## First optimization


The main difference for this first optimization is that the batch size has been set to = -1 

where the system can dynamically change based on GPU availability. This is to test the biggest possible batch size and any effects it has

In [8]:
model2 = YOLO('yolov8s.pt')
model2.train(data="config.yaml",epochs=50,patience=5,batch=-1, lr0=0.5,imgsz=640)
model2.to('cuda')

Ultralytics YOLOv8.1.27 🚀 Python-3.10.13 torch-2.2.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3070, 8192MiB)
engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=config.yaml, epochs=50, time=None, patience=5, batch=-1, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_bo

train: Scanning C:\Users\Pocholo\Desktop\481\CV_Project\labels\train.cache... 1522 images, 129 backgrounds, 0 corrupt: 100%|██████████| 1522/1522 [00:00<?, ?it/s]
val: Scanning C:\Users\Pocholo\Desktop\481\CV_Project\labels\val.cache... 118 images, 9 backgrounds, 0 corrupt: 100%|██████████| 118/118 [00:00<?, ?it/s]


Plotting labels to runs\detect\train2\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.5' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.00046875), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train2
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/50      5.11G      1.412      2.879      1.599         15        640: 100%|██████████| 77/77 [00:18<00:00,  4.23it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  2.66it/s]

                   all        118        444      0.535      0.597      0.585      0.356


...

                   all        118        444      0.848      0.779      0.865      0.562
Stopping training early as no improvement observed in last 5 epochs. Best results observed at epoch 28, best model saved as best.pt.
To update EarlyStopping(patience=5) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



33 epochs completed in 0.181 hours.
Optimizer stripped from runs\detect\train2\weights\last.pt, 22.5MB
Optimizer stripped from runs\detect\train2\weights\best.pt, 22.5MB

Validating runs\detect\train2\weights\best.pt...
Ultralytics YOLOv8.1.27 🚀 Python-3.10.13 torch-2.2.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3070, 8192MiB)
Model summary (fused): 168 layers, 11127906 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  2.64it/s]


                   all        118        444      0.844      0.814      0.888      0.578
                  Dime        118         50      0.772       0.72        0.8      0.461
                Loonie        118         56      0.851      0.946      0.943      0.651
                Nickel        118         61      0.856      0.738      0.882      0.626
                 Penny        118         83      0.838      0.812      0.891      0.601
               Quarter        118        114      0.826      0.693      0.857      0.478
                Toonie        118         80       0.92      0.975      0.957      0.647
Speed: 0.1ms preprocess, 3.7ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs\detect\train2


YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(96, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_s

## Second Optimization

For the second optimization, we doubled the original batch size from 8 -> 16

This will be the middle ground between the baseline of 8 and the max of -1

In [4]:
model3 = YOLO('yolov8s.pt')
model3.train(data="config.yaml",epochs=50,patience=5,batch=16, lr0=0.5,imgsz=640)
model3.to('cuda')

Ultralytics YOLOv8.1.27 🚀 Python-3.10.13 torch-2.2.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3070, 8192MiB)
engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=config.yaml, epochs=50, time=None, patience=5, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_bo

train: Scanning C:\Users\Pocholo\Desktop\481\CV_Project\labels\train.cache... 1522 images, 129 backgrounds, 0 corrupt: 100%|██████████| 1522/1522 [00:00<?, ?it/s]
val: Scanning C:\Users\Pocholo\Desktop\481\CV_Project\labels\val.cache... 118 images, 9 backgrounds, 0 corrupt: 100%|██████████| 118/118 [00:00<?, ?it/s]


Plotting labels to runs\detect\train4\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.5' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train4
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/50      4.35G      1.413       2.71      1.579          9        640: 100%|██████████| 96/96 [00:18<00:00,  5.15it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:00<00:00,  4.03it/s]

                   all        118        444      0.452      0.608      0.536      0.308


...

                   all        118        444      0.816      0.803      0.858      0.555
Stopping training early as no improvement observed in last 5 epochs. Best results observed at epoch 26, best model saved as best.pt.
To update EarlyStopping(patience=5) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



31 epochs completed in 0.160 hours.
Optimizer stripped from runs\detect\train4\weights\last.pt, 22.5MB
Optimizer stripped from runs\detect\train4\weights\best.pt, 22.5MB

Validating runs\detect\train4\weights\best.pt...
Ultralytics YOLOv8.1.27 🚀 Python-3.10.13 torch-2.2.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3070, 8192MiB)
Model summary (fused): 168 layers, 11127906 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  3.52it/s]


                   all        118        444      0.747       0.88      0.867      0.566
                  Dime        118         50      0.643       0.76      0.748      0.439
                Loonie        118         56       0.76      0.964      0.871      0.612
                Nickel        118         61      0.615      0.918      0.867      0.635
                 Penny        118         83      0.794      0.867      0.895      0.581
               Quarter        118        114      0.781      0.789      0.838      0.458
                Toonie        118         80      0.887      0.979      0.985      0.672
Speed: 0.2ms preprocess, 3.5ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to runs\detect\train4


YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(96, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_s